## Control v.s. Treatment

In [1]:
import os
import pandas as pd
import pingouin as pt
import warnings

warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.float_format', '{:.3f}'.format)

### ```拆分成 control 與 treatment```

In [4]:
merge = pd.read_csv(r'C:\Users\User\Desktop\poll\data\processed_data\merge_data.csv', index_col=0)
control = merge.loc[(merge['treatment'] == 0)]
treatment = merge.loc[(merge['treatment'] == 1)]

In [7]:
# control wtp
control_wtp = control[['player.wtp_voting_cost', 'player.wtp_voting_cost_pq', 'sb_condition']]
control_wtp['wtp_sum'] = control_wtp.apply(lambda x : x['player.wtp_voting_cost'] + x['player.wtp_voting_cost_pq'], axis = 1)

# treatment wtp
treatment_wtp = treatment[['player.wtp_voting_cost', 'sb_condition']]
treatment_wtp.rename(columns = {'player.wtp_voting_cost':'wtp'}, inplace = True)

### ```拆分成大大、小小、大小、小大```

In [8]:
# select data based on sb_condition
bb = 'bb'
bb_t = treatment_wtp.query('sb_condition == @bb')
bb_c = control_wtp.query('sb_condition == @bb')

ss = 'ss'
ss_t = treatment_wtp.query('sb_condition == @ss')
ss_c = control_wtp.query('sb_condition == @ss')

sb = 'sb'
sb_t = treatment_wtp.query('sb_condition == @sb')
sb_c = control_wtp.query('sb_condition == @sb')

bs = 'bs'
bs_t = treatment_wtp.query('sb_condition == @bs')
bs_c = control_wtp.query('sb_condition == @bs')

In [9]:
# 顯示人數分配
print('control group:')
print(f"【大大】: {len(bb_c)}")
print(f"【大小】: {len(bs_c)}")
print(f"【小大】: {len(sb_c)}")
print(f"【小小】: {len(ss_c)}")

print('---------------------')

print('treatment group:')
print(f"【大大】: {len(bb_t)}")
print(f"【大小】: {len(bs_t)}")
print(f"【小大】: {len(sb_t)}")
print(f"【小小】: {len(ss_t)}")

control group:
【大大】: 135
【大小】: 105
【小大】: 105
【小小】: 105
---------------------
treatment group:
【大大】: 135
【大小】: 105
【小大】: 105
【小小】: 105


### ```all```

In [10]:
# implement unpaired ttest
pt.ttest(control_wtp['wtp_sum'], treatment_wtp['wtp'], paired = False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.495,898,two-sided,0.135,"[-0.72, 5.36]",0.100,0.224,0.321


In [11]:
# get the mean of wtp
print('control 與 treatment 的 wtp 平均值')
print('---------------------------------')
print(f"control: {control_wtp['wtp_sum'].describe()['mean']}")
print(f"treamtent : {treatment_wtp['wtp'].describe()['mean']}")

control 與 treatment 的 wtp 平均值
---------------------------------
control: 29.788888888888888
treamtent : 27.47111111111111


### ```bb```

In [12]:
# bb
pt.ttest(bb_c['wtp_sum'], bb_t['wtp'], paired = False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.095,268,two-sided,0.275,"[-2.82, 9.87]",0.133,0.236,0.194


In [13]:
# get the mean of wtp
print('【大大】control 與 treatment 的 wtp 平均值')
print('---------------------------------')
print(f"control: {bb_c['wtp_sum'].describe()['mean']}")
print(f"treamtent: {bb_t['wtp'].describe()['mean']}")

【大大】control 與 treatment 的 wtp 平均值
---------------------------------
control: 39.77777777777778
treamtent: 36.25185185185185


### ```ss```

In [14]:
# ss
pt.ttest(ss_c['wtp_sum'], ss_t['wtp'], paired = False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.275,208,two-sided,0.204,"[-1.86, 8.66]",0.176,0.322,0.245


In [15]:
# get the mean of wtp
print('【小小】control 與 treatment 的 wtp 平均值')
print('---------------------------------')
print(f"control: {ss_c['wtp_sum'].describe()['mean']}")
print(f"treamtent: {ss_t['wtp'].describe()['mean']}")

【小小】control 與 treatment 的 wtp 平均值
---------------------------------
control: 20.552380952380954
treamtent: 17.152380952380952


### ```sb```

In [16]:
# sb
pt.ttest(sb_c['wtp_sum'], sb_t['wtp'], paired = False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.789,208,two-sided,0.431,"[-8.7, 3.73]",0.109,0.201,0.123


In [17]:
# get the mean of wtp
print('【小大】control 與 treatment 的 wtp 平均值')
print('---------------------------------')
print(f"control: {sb_c['wtp_sum'].describe()['mean']}")
print(f"treamtent: {sb_t['wtp'].describe()['mean']}")

【小大】control 與 treatment 的 wtp 平均值
---------------------------------
control: 28.133333333333333
treamtent: 30.61904761904762


### ```bs```

In [18]:
# bs
pt.ttest(bs_c['wtp_sum'], bs_t['wtp'], paired = False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.852,208,two-sided,0.065,"[-0.29, 9.26]",0.256,0.747,0.454


In [19]:
# get the mean of wtp
print('【大小】control 與 treatment 的 wtp 平均值')
print('---------------------------------')
print(f"control: {bs_c['wtp_sum'].describe()['mean']}")
print(f"treamtent: {bs_t['wtp'].describe()['mean']}")

【大小】control 與 treatment 的 wtp 平均值
---------------------------------
control: 27.83809523809524
treamtent: 23.35238095238095
